In [3]:
%load_ext autoreload
%autoreload 2

import os 
# os.environ['R_HOME']= r'C:\Users\tomha\miniconda3\envs\octagon_analysis\lib\R'
# os.environ['R_HOME']= r'D:\Users\Tom\miniconda3\envs\octagon_analysis\lib\R'
os.environ['R_HOME']= '/home/tom/miniconda3/envs/octagon_analysis/lib/R'

import rpy2

import rpy2.robjects as robjects
print(robjects.r('R.version.string'))

import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import globals
import data_strings
import data_extraction.get_indices as get_indices
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
from trajectory_analysis import trajectory_vectors
from plotting import plot_octagon
import parse_data.identify_filepaths as identify_filepaths 
from data_extraction.trial_list_filters import filter_trials_other_visible
from analysis import opponent_visibility
from ipywidgets import IntProgress
from IPython.display import display
import time
from pymer4.models import Lmer
import populate_dataframes



[1] "R version 4.3.3 (2024-02-29)"



In [4]:
k = 20
split_dataframes = True

### load data

In [5]:
import pickle

analysis_dir = os.path.join('..', 'data')
analysis_file = 'analysis_results_2levelsforFirstSeenWall.pkl'
filename = os.path.join(analysis_dir, analysis_file)
# load the analysis results
with open(filename, 'rb') as f:
    analysis_results = pickle.load(f)

### populate dataframes for glm input

In [6]:
# populate dataframes for solo, solosocial, and social analysis_type
glm_df_solo = populate_dataframes.populate_dataframe(analysis_results, analysis_type='solo')
glm_df_solosocial = populate_dataframes.populate_dataframe(analysis_results, analysis_type='solosocial')
glm_df_social = populate_dataframes.populate_dataframe(analysis_results, analysis_type='social')


### create reference to dataframes

In [7]:
dataframes = {
    'glm_df_solo': glm_df_solo,
    'glm_df_solosocial': glm_df_solosocial,
    'glm_df_social': glm_df_social
}

### shuffle the dataframes for k-fold index selection


In [8]:
shuffled_dataframes = os.path.join('..', 'data', 'k_fold_CV', 'shuffled_dataframes.pkl')

if split_dataframes:
    
    # shuffle each dataframe
    for name, df in dataframes.items():
        dataframes[name] = df.sample(frac=1, random_state=17).reset_index(drop=True)

    # pickle save shuffled dataframes to sandbox > data, as one dictionary
    with open(shuffled_dataframes, 'wb') as f:
        pickle.dump(dataframes, f)

else:
    # load the shuffled dataframes
    with open(shuffled_dataframes, 'rb') as f:
        shuffled_dataframes = pickle.load(f)



### Split each dataframe into folds

In [9]:
# split each dataframe into k equal parts
split_dataframes = {name: np.array_split(df, k) for name, df in dataframes.items()}

/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [21]:
type(split_dataframes['glm_df_solo'][0])

pandas.core.frame.DataFrame

### Cross-validation functions

In [ ]:
from contextlib import redirect_stdout

def fit_models(split_df, model_formula):
    '''
    Takes a dataframe and a model formula, and fits k models to the data.
    Wjere k is the number of folds in the split dataframe.
    Use k-1 folds to train each model'''
    
    models = []
    max_count = len(split_df)
    f = IntProgress(min=0, max=max_count, description='Fitting models')
    display(f)

    for i, df in enumerate(split_df):
        print(f"Fold {i}: Type = {type(df)}")

    # Suppress the output of the models fitting process
    with open(os.devnull, 'w') as fnull:
        with redirect_stdout(fnull):
            for i in range(len(split_df)):
                # Combine all folds except the i-th fold
                train_data = pd.concat([df for j, df in enumerate(split_df) if j != i], ignore_index=True)                
                # Fit the Lmer model to these folds
                model = Lmer(model_formula, data=train_data, family='binomial')
                model.fit()
                models.append(model)
                print(f"Model {i} fit with {len(train_data)} rows")
                f.value += 1
    
    return models


def calculate_predictions(split_df, models):
    ''' 
    Predict on the held-out fold, for each trained model
    '''

    all_predictions = []
    
    # for each dataframe type, predict on the held-out fold using the relevant model
    for i, model in enumerate(models):
        
        # predict on the held-out fold
        predict_data = split_df[i]
        predictions = model.predict(predict_data, skip_data_checks=True, verify_predictions=False)

        # concatenate these predictions to the predictions array
        all_predictions.append(predictions)

    return all_predictions



def calculate_likelihoods(split_df, predictions):
    ''' 
    Calculate the likelihood of each prediction given the true output.
    The likelihood is calculated as p^y * (1-p)^(1-y), where p is the predicted probability
    and y is the true output (0 or 1).

    Takes a list of dataframes and a list of predictions, where each array of predictions corresponds
    to the dataframe of the same index.
    '''
    
    all_likelihoods = []
    for i, prediction_fold in enumerate(predictions):
        likelihoods = np.full(len(prediction_fold), np.nan)
        predicted_output = prediction_fold
        true_output = split_df[i]
        
        # calculate the metric for each prediction
        for i, prediction in enumerate(predicted_output):
            ground_truth = true_output.iloc[i]['ChooseHigh']
            likelihood = prediction**ground_truth * (1 - prediction)**(1 - ground_truth)
            likelihoods[i] = likelihood

        all_likelihoods.append(likelihoods)
    
    return all_likelihoods

def calculate_nll(likelihoods):
    # #### sum the logs of the likelihoods, and take the negative

    concatenated_likelihoods = np.concatenate(likelihoods)
    summed_log_likelihoods = np.sum(np.log(concatenated_likelihoods)) 
    nll = -summed_log_likelihoods
    average_nll = nll / concatenated_likelihoods.shape[0]

    return nll, average_nll


def save_cross_validation_results(name, model_formula, split_df, num_folds, predictions, nll, average_nll, k):
    ''' Save the cross-validation results to a file. '''
    
    cross_validation_results = {
        "name": name,
        "model_formula": model_formula,
        "split_df": split_df,
        "num_folds" : num_folds,
        # "models" : models,
        "predictions" : predictions,
        "nll" : nll,
        "average_nll" : average_nll
    }

   # Save the cross-validation results to a file
    dir = os.path.join('..', 'data', 'k_fold_CV')
    filename = f'{k}-fold-CV_results_{name}.pickle'
    filepath = os.path.join(dir, filename)
    with open(filepath, 'wb') as f:
        pickle.dump(cross_validation_results, f)

    print(f"{k}-fold CV data saved to: ", filepath)

In [14]:
def run_cross_validation(split_df, model_formula, name, save_results=False):
    ''' 
    Run k-fold cross-validation on the given dataframes.
    '''

    n_folds = len(split_df)

    # Step 1: Fit models on k-1 folds for all iterations
    models = fit_models(split_df, model_formula)

    # Step 2: Calculate predictions on the held-out fold for each model
    predictions = calculate_predictions(split_df, models)

    # Step 3: Calculate likelihoods for each prediction
    likelihoods = calculate_likelihoods(split_df, predictions)

    # Step 4: Calculate NLL
    nll, average_nll = calculate_nll(likelihoods)

    # Step 5: Save data to file (optional)
    if save_results:
        save_cross_validation_results(name, model_formula, split_df, n_folds, predictions, nll, average_nll, k=n_folds)

    return nll, models, predictions, likelihoods


In [24]:
# load the file '20-fold-CV_results_solo_randomintercepts.pickle' from data/k_fold_CV
with open(os.path.join('..', 'data', 'k_fold_CV', '20-fold-CV_results_solo_randomintercepts.pickle'), 'rb') as f:
    cross_validation_results = pickle.load(f)

In [27]:
cross_validation_results['average_nll']

np.float64(0.254092780233656)

In [22]:
model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + (1|GlmPlayerID)'
(nll, models,
  predictions, likelihoods) = run_cross_validation(split_dataframes['glm_df_solo'], model_formula,
                                                    "solo_randomintercepts",
                                                      save_results=True)

IntProgress(value=0, description='Fitting models', max=20)

Fold 0: Type = <class 'pandas.core.frame.DataFrame'>
Fold 1: Type = <class 'pandas.core.frame.DataFrame'>
Fold 2: Type = <class 'pandas.core.frame.DataFrame'>
Fold 3: Type = <class 'pandas.core.frame.DataFrame'>
Fold 4: Type = <class 'pandas.core.frame.DataFrame'>
Fold 5: Type = <class 'pandas.core.frame.DataFrame'>
Fold 6: Type = <class 'pandas.core.frame.DataFrame'>
Fold 7: Type = <class 'pandas.core.frame.DataFrame'>
Fold 8: Type = <class 'pandas.core.frame.DataFrame'>
Fold 9: Type = <class 'pandas.core.frame.DataFrame'>
Fold 10: Type = <class 'pandas.core.frame.DataFrame'>
Fold 11: Type = <class 'pandas.core.frame.DataFrame'>
Fold 12: Type = <class 'pandas.core.frame.DataFrame'>
Fold 13: Type = <class 'pandas.core.frame.DataFrame'>
Fold 14: Type = <class 'pandas.core.frame.DataFrame'>
Fold 15: Type = <class 'pandas.core.frame.DataFrame'>
Fold 16: Type = <class 'pandas.core.frame.DataFrame'>
Fold 17: Type = <class 'pandas.core.frame.DataFrame'>
Fold 18: Type = <class 'pandas.core.fr

/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprec

20-fold CV data saved to:  ../data/k_fold_CV/20-fold-CV_results_solo_randomintercepts.pickle


In [28]:
model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + OpponentVisible + OpponentD2H' \
' + OpponentD2L + OpponentFirstSeenWall + (1|GlmPlayerID)'

(nll, models,
  predictions, likelihoods) = run_cross_validation(split_dataframes['glm_df_social'], model_formula,
                                                    "social_randomintercepts_opponentvisible",
                                                      save_results=True)

IntProgress(value=0, description='Fitting models', max=20)

Fold 0: Type = <class 'pandas.core.frame.DataFrame'>
Fold 1: Type = <class 'pandas.core.frame.DataFrame'>
Fold 2: Type = <class 'pandas.core.frame.DataFrame'>
Fold 3: Type = <class 'pandas.core.frame.DataFrame'>
Fold 4: Type = <class 'pandas.core.frame.DataFrame'>
Fold 5: Type = <class 'pandas.core.frame.DataFrame'>
Fold 6: Type = <class 'pandas.core.frame.DataFrame'>
Fold 7: Type = <class 'pandas.core.frame.DataFrame'>
Fold 8: Type = <class 'pandas.core.frame.DataFrame'>
Fold 9: Type = <class 'pandas.core.frame.DataFrame'>
Fold 10: Type = <class 'pandas.core.frame.DataFrame'>
Fold 11: Type = <class 'pandas.core.frame.DataFrame'>
Fold 12: Type = <class 'pandas.core.frame.DataFrame'>
Fold 13: Type = <class 'pandas.core.frame.DataFrame'>
Fold 14: Type = <class 'pandas.core.frame.DataFrame'>
Fold 15: Type = <class 'pandas.core.frame.DataFrame'>
Fold 16: Type = <class 'pandas.core.frame.DataFrame'>
Fold 17: Type = <class 'pandas.core.frame.DataFrame'>
Fold 18: Type = <class 'pandas.core.fr

/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprec

20-fold CV data saved to:  ../data/k_fold_CV/20-fold-CV_results_social_randomintercepts_opponentvisible.pickle
